## Libraries

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

## Dateset and Parameters

In [39]:
base_dir = "./dataset"
train_dir = os.path.join(base_dir, 'training')
test_dir = os.path.join(base_dir, 'testing')
categories = ['glioma', 'meningioma', 'notumor', 'pituitary']

img_size = 128
batch_size = 32
epochs = 50

## Data Preparation

In [40]:
def prepare_data(directory, categories, img_size):
    data = []
    labels = []
    for category in categories:
        path = os.path.join(directory, category)
        class_num = categories.index(category)
        for img in os.listdir(path):
            try:
                img_array = tf.keras.preprocessing.image.load_img(os.path.join(path, img), target_size=(img_size, img_size), color_mode="grayscale")
                img_array = tf.keras.preprocessing.image.img_to_array(img_array)
                img_array = img_array / 255.0
                data.append(img_array)
                labels.append(class_num)
            except Exception as e:
                pass
    return np.array(data), np.array(labels)

# Load and preprocess data
X_train, y_train = prepare_data(train_dir, categories, img_size)
X_test, y_test = prepare_data(test_dir, categories, img_size)
y_train = to_categorical(y_train, num_classes=len(categories))
y_test = to_categorical(y_test, num_classes=len(categories))

### Data Augmentation

In [41]:
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)
train_generator = datagen.flow(X_train, y_train, batch_size=batch_size)


### Capsule Network Model

In [67]:
class CapsuleLayer(layers.Layer):
    def __init__(self, num_capsules, dim_capsules, routings=3, **kwargs):
        super(CapsuleLayer, self).__init__(**kwargs)
        self.num_capsules = num_capsules
        self.dim_capsules = dim_capsules
        self.routings = routings
        self.kernel = None

    def build(self, input_shape):
        assert len(input_shape) == 3, "Input shape should be [None, input_capsule_num, input_capsule_dim]"
        self.input_capsule_num = input_shape[1]
        self.input_capsule_dim = input_shape[2]
        self.kernel = self.add_weight(shape=(self.input_capsule_num, self.num_capsules, self.input_capsule_dim, self.dim_capsules),
                                      initializer='glorot_uniform',
                                      name='capsule_kernel')

    def call(self, inputs, training=None):
        inputs_expand = tf.expand_dims(tf.expand_dims(inputs, 2), -1)
        inputs_tiled = tf.tile(inputs_expand, [1, 1, self.num_capsules, 1, 1])
        inputs_hat = tf.reduce_sum(inputs_tiled * self.kernel, axis=3)
        
        b = tf.zeros(shape=[tf.shape(inputs)[0], self.input_capsule_num, self.num_capsules, 1])
        for i in range(self.routings):
            c = tf.nn.softmax(b, axis=2)
            outputs = tf.reduce_sum(c * inputs_hat, axis=1, keepdims=True)
            outputs = tf.squeeze(tf.nn.l2_normalize(outputs, -1), axis=1)
            if i < self.routings - 1:
                b += tf.reduce_sum(inputs_hat * tf.expand_dims(outputs, 1), -1, keepdims=True)
        
        return outputs

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.num_capsules, self.dim_capsules)

class Length(layers.Layer):
    def call(self, inputs, **kwargs):
        return tf.sqrt(tf.reduce_sum(tf.square(inputs), -1) + 1e-9)

### Define the CapsNet model

In [68]:
input_image = layers.Input(shape=(img_size, img_size, 1))
conv1 = layers.Conv2D(256, (9, 9), activation='relu')(input_image)
conv2 = layers.Conv2D(256, (9, 9), strides=(2, 2), activation='relu')(conv1)
primary_caps = layers.Conv2D(32 * 8, (9, 9), activation='relu')(conv2)

print("Shape of primary_caps:", primary_caps.shape)

Shape of primary_caps: (None, 48, 48, 256)


### Adjust reshape layer based on printed shape

In [69]:
primary_caps_reshape = layers.Reshape((-1, 256))(primary_caps)
capsule = CapsuleLayer(num_capsules=len(categories), dim_capsules=16)(primary_caps_reshape)
output = Length()(capsule)

model = models.Model(inputs=input_image, outputs=output)
model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "functional_33"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_14 (InputLayer)     │ (None, 128, 128, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_42 (Conv2D)              │ (None, 120, 120, 256)  │        20,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_43 (Conv2D)              │ (None, 56, 56, 256)    │     5,308,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_44 (Conv2D)              │ (None, 48, 48, 256)    │     5,308,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_14 (Reshape)            │ (None, 2304, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ capsule_layer_16 (CapsuleLayer) │ (None, 4, 16)          │    37,748,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ length_16 (Length)              │ (None, 4)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 48,387,072 (184.58 MB)

 Trainable params: 48,387,072 (184.58 MB)

 Non-trainable params: 0 (0.00 B)

## Training the Model

In [70]:
history = model.fit(train_generator, steps_per_epoch=len(X_train) // batch_size, epochs=epochs, validation_data=(X_test, y_test))

Epoch 1/50


c:\Users\Lester\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


ResourceExhaustedError: Graph execution error:

Detected at node gradient_tape/functional_33_1/capsule_layer_16_1/mul/Mul_1 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\Lester\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "C:\Users\Lester\AppData\Roaming\Python\Python312\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "C:\Users\Lester\AppData\Roaming\Python\Python312\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "C:\Users\Lester\AppData\Roaming\Python\Python312\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "c:\Users\Lester\AppData\Local\Programs\Python\Python312\Lib\asyncio\base_events.py", line 641, in run_forever

  File "c:\Users\Lester\AppData\Local\Programs\Python\Python312\Lib\asyncio\base_events.py", line 1987, in _run_once

  File

### Save the model

In [ ]:
model.save("capsnet_brain_tumor_model.h5")

### Evaluate the model

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

### Generate classification report

In [ ]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes, target_names=categories))

### Confusion matrix

In [ ]:
conf_matrix = confusion_matrix(y_true, y_pred_classes)
print(conf_matrix)